In [ ]:
from pymongo import MongoClient

def insert_data():
    client = MongoClient('mongodb://localhost:27017/')
    db = client.local  # 'local' 데이터베이스 사용

    # 책 데이터 삽입
    books = [
        {"title": "Kafka on the Shore", "author": "Haruki Murakami", "year": 2002},
        {"title": "Norwegian Wood", "author": "Haruki Murakami", "year": 1987},
        {"title": "1Q84", "author": "Haruki Murakami", "year": 2009}
    ]
    db.books.insert_many(books)

    # 영화 데이터 삽입
    movies = [
        {"title": "Inception", "director": "Christopher Nolan", "year": 2010, "rating": 8.8, "genre": "fantasy"},
        {"title": "Interstellar", "director": "Christopher Nolan", "year": 2014, "rating": 8.6, "genre": "scifi"},
        {"title": "The Dark Knight", "director": "Christopher Nolan", "year": 2008, "rating": 9.0, "genre": "action"}
    ]
    db.movies.insert_many(movies)

    # 사용자 행동 데이터 삽입
    user_actions = [
        {"user_id": 1, "action": "click", "timestamp": "2023-04-12T08:00:00Z"},
        {"user_id": 1, "action": "view", "timestamp": "2023-04-12T09:00:00Z"},
        {"user_id": 2, "action": "purchase", "timestamp": "2023-04-12T10:00:00Z"}
    ]
    db.user_actions.insert_many(user_actions)

    print("Data inserted successfully")
    client.close()

if __name__ == "__main__":
    insert_data()


In [71]:
# - **문제 설명**:
# 사용자는 "fantasy" 장르의 모든 책을 찾고 싶어합니다.
# - **쿼리 작성 목표**:
# "fantasy" 장르에 해당하는 모든 책의 제목과 저자를 찾는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.
def find_genre(db,genre):
    collection = db.movies
    query = {"genre": genre}
    projection = {"_id": 0, "director": 1, "year": 1, "rating": 1}

    movies = collection.find(query, projection)
    for movie in movies:
        print(movie)

client = MongoClient('mongodb://localhost:27017/')
find_genre(client.local,'fantasy')

{'director': 'Christopher Nolan', 'year': 2010, 'rating': 8.8}
{'director': 'Christopher Nolan', 'year': 2010, 'rating': 8.8}
{'director': 'Christopher Nolan', 'year': 2010, 'rating': 8.8}
{'director': 'Christopher Nolan', 'year': 2010, 'rating': 8.8}


In [85]:
# - **문제 설명**:
# 각 영화 감독별로 그들의 영화 평점의 평균을 계산하고 싶습니다. 이를 통해 어떤 감독이 가장 높은 평균 평점을 가지고 있는지 알아볼 수 있습니다.
# - **쿼리 작성 목표**:
# 모든 영화 감독의 영화 평점 평균을 계산하고, 평균 평점이 높은 순으로 정렬하는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.

from pymongo import MongoClient

def calculate_average_rating(db):
    collection = db.movies

    pipeline = [
        {
            "$group": {
                "_id": None,
                "average_rating": {"$avg": "$rating"}
            }
        }
    ]

    result = list(collection.aggregate(pipeline))
    print(result)


if __name__ == "__main__":
    client = MongoClient('mongodb://localhost:27017/')
    calculate_average_rating(client.local)
    client.close()


[{'_id': None, 'average_rating': 8.8}]


In [93]:
# - **문제 설명**:
# 특정 사용자의 최근 행동 로그를 조회하고자 합니다. 이 때, 최신 순으로 정렬하여 최근 5개의 행동만을 보고 싶습니다.
# - **쿼리 작성 목표**:
# 사용자 ID가 1인 사용자의 최근 행동 5개를 시간 순으로 정렬하여 조회하는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.
def find_recent_actions_by_user(db, user_id, limit=5):
    u_a_collection = db.user_actions
    query = {"user_id": user_id}
    sort_criteria = [("timestamp", -1)]

    actions = u_a_collection.find(query).sort(sort_criteria).limit(limit)
    for action in actions:
        print(action)

client = MongoClient('mongodb://localhost:27017/')
find_recent_actions_by_user(client.local, 1)



{'_id': ObjectId('662a04122b600b62447b1099'), 'user_id': 1, 'action': 'view', 'timestamp': '2023-04-12T09:00:00Z'}
{'_id': ObjectId('662a0f6f2b600b62447b10ad'), 'user_id': 1, 'action': 'view', 'timestamp': '2023-04-12T09:00:00Z'}
{'_id': ObjectId('662a0a632b600b62447b10a3'), 'user_id': 1, 'action': 'view', 'timestamp': '2023-04-12T09:00:00Z'}
{'_id': ObjectId('662a02972b600b62447b1085'), 'user_id': 1, 'action': 'view', 'timestamp': '2023-04-12T09:00:00Z'}
{'_id': ObjectId('662a040f2b600b62447b108f'), 'user_id': 1, 'action': 'view', 'timestamp': '2023-04-12T09:00:00Z'}


In [95]:
# - **문제 설명** :
# 데이터베이스에 저장된 책 데이터를 이용하여 각 출판 연도별로 출판된 책의 수를 계산하고자 합니다. 이 데이터는 시간에 따른 출판 트렌드를 분석하는 데 사용될 수 있습니다.
# - **쿼리 작성 목표** :
# 각 출판 연도별로 출판된 책의 수를 계산하고, 출판된 책의 수가 많은 순서대로 정렬하는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.
#파이프라인은 아직도 모르겟군
def count_books_by_year(db):
    books_collection = db.books
    pipeline = [
        {"$group": {"_id": "$year", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}}
    ]

    results = books_collection.aggregate(pipeline)
    for result in results:
        print(result)

#함수 실행 코드
client = MongoClient('mongodb://localhost:27017/')
count_books_by_year(client.local)


{'_id': 2002, 'count': 15}
{'_id': 1987, 'count': 15}
{'_id': 2009, 'count': 15}


In [96]:
# - **문제 설명**:
# 특정 사용자의 행동 로그 중, 특정 날짜 이전의 "view" 행동을 "seen"으로 변경하고 싶습니다. 예를 들어, 사용자 ID가 1인 사용자의 2023년 4월 10일 이전의 모든 "view" 행동을 "seen"으로 변경하는 작업입니다.
# - **쿼리 작성 목표**:
# 사용자 ID가 1인 사용자의 2023년 4월 10일 이전의 "view" 행동을 "seen"으로 변경하는 MongoDB 업데이트 쿼리를 함수로 만들어 문제를 해결해 봅니다.
# 모르겠군
from datetime import datetime

def update_user_actions_before_date(db, user_id, date, old_action, new_action):
    user_actions_collection = db.user_actions
    query = {"user_id": user_id, "action": old_action, "timestamp": {"$lt": date}}
    update = {"$set": {"action": new_action}}

    result = user_actions_collection.update_many(query, update)
    print(f"Updated {result.modified_count} documents.")

#함수 실행 코드
client = MongoClient('mongodb://localhost:27017/')
update_user_actions_before_date(client.local, 1, datetime(2023, 4, 10), "view", "seen")

Updated 0 documents.
